In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
def get_aggregate_df(market_data, ticker, zoom=1):
    def get_timezoom(x, zoom):
        if x[1] != ':':
            t = int(x[:2])
        else:
            t = int(x[0])
        return str(t // zoom)

    ticker_market = market_data[market_data['SYM_ROOT'] == ticker]
    ticker_market['timezoom'] = ticker_market['TIME_M'].apply(lambda x: get_timezoom(x, zoom))
    ticker_market['createdate'] = ticker_market['DATE'].apply(lambda x: str(x)) + ' ' + ticker_market['timezoom']
    ticker_market = ticker_market[['SIZE', 'PRICE', 'createdate']]

    ticker_size = ticker_market.groupby(['createdate']).sum()['SIZE']
    ticker_high = ticker_market.groupby(['createdate']).max()['PRICE']
    ticker_low = ticker_market.groupby(['createdate']).min()['PRICE']
    ticker_open = ticker_market.groupby(['createdate'])['PRICE'].apply(lambda x: x.iloc[0])
    ticker_close = ticker_market.groupby(['createdate'])['PRICE'].apply(lambda x: x.iloc[-1])
    ticker_market_zoom = pd.DataFrame([ticker_size, ticker_high, ticker_low, ticker_open, ticker_close]).T
    ticker_market_zoom.columns = ['volume', 'high', 'low', 'open', 'close']

    ticker_market_zoom['volatility'] = 2 * (ticker_market_zoom['high'] - ticker_market_zoom['low']) / (
                ticker_market_zoom['low'] + ticker_market_zoom['high'])
    ticker_market_zoom['return'] = np.log(ticker_market_zoom['close']) / np.log(ticker_market_zoom['open'])

    spy = pd.read_csv('SPY.csv')
    spy['timezoom'] = spy['TIME_M'].apply(lambda x: get_timezoom(x, zoom))
    spy['createdate'] = spy['DATE'].apply(lambda x: str(x)) + ' ' + spy['timezoom']
    spy_zoom = pd.DataFrame(columns=['close', 'open'])
    spy_zoom['open'] = spy.groupby(['createdate'])['PRICE'].apply(lambda x: x.iloc[0])
    spy_zoom['close'] = spy.groupby(['createdate'])['PRICE'].apply(lambda x: x.iloc[-1])
    spy_zoom['return'] = np.log(spy_zoom['close']) / np.log(spy_zoom['open'])

    ticker_market_zoom['excess_return'] = ticker_market_zoom['return'] - spy_zoom['return']
    # ticker_market_zoom = ticker_market_zoom[['volume', 'volatility', 'excess_return']]

    ticker_tweet = pd.read_csv('./tweets/' + ticker + '.csv', index_col=0)
    ticker_tweet['createdate'] = ticker_tweet['createdate'].apply(
        lambda x: x[:4] + x[5:7] + x[8:10] + ' ' + str(int(x[12]) // zoom))
    ticker_tweet_groupby = ticker_tweet.groupby(['createdate', 'sentiment']).size().unstack()

    ticker = ticker_market_zoom.merge(ticker_tweet_groupby, on='createdate', how='outer').iloc[3:, :]

    days = ['20201130', '20201201', '20201202', '20201203', '20201204', '20201207', '20201208']
    zooms = [i for i in range(int(24 / zoom))]
    x = []
    for day in days:
        for t in zooms:
            x.append(day + ' ' + str(t))
    empty_df = pd.DataFrame(index=x)
    empty_df.index.name = 'createdate'
    ticker = empty_df.merge(ticker, on='createdate', how='left').sort_index()
    return ticker

In [3]:
market_data = pd.read_csv('market_data.csv')
ticker = 'CSCO'
ticker_df = get_aggregate_df(market_data, ticker, zoom=1)
ticker_df = ticker_df.dropna()

In [6]:
ticker_df['excess_return'] = (ticker_df['excess_return']-np.mean(ticker_df['excess_return']))/np.std(ticker_df['excess_return'])
ticker_df['excess_return'] = (ticker_df['volatility']-np.mean(ticker_df['volatility']))/np.std(ticker_df['volatility'])
ticker_df['volume'] = (ticker_df['volume']-np.mean(ticker_df['volume']))/np.std(ticker_df['volume'])

In [7]:
ticker_df['G'] = ticker_df['positive'].shift(1)
ticker_df['B'] = ticker_df['negative'].shift(1)
ticker_df['ER1'] = ticker_df['excess_return'].shift(1)
ticker_df['ER2'] = ticker_df['excess_return'].shift(2)
ticker_df['VOL1'] = ticker_df['volatility'].shift(1)
ticker_df['VOL2'] = ticker_df['volatility'].shift(2)
ticker_df['volume1'] = ticker_df['volume'].shift(1)
ticker_df['volume2'] = ticker_df['volume'].shift(2)
ticker_df = ticker_df.dropna()
ticker_df['SR'] = (ticker_df['G']-ticker_df['B'])/(ticker_df['G']+ticker_df['B'])
ticker_df['const'] = 1

In [8]:
import statsmodels.api as sm
reg1 = sm.OLS(endog=ticker_df['excess_return'], exog=ticker_df[['ER1', 'ER2', 'G', 'B', 'SR', 'const']], missing='drop')
results = reg1.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          excess_return   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                 -0.115
Method:                 Least Squares   F-statistic:                    0.2182
Date:                Fri, 11 Dec 2020   Prob (F-statistic):              0.952
Time:                        00:04:35   Log-Likelihood:                -55.534
No. Observations:                  39   AIC:                             123.1
Df Residuals:                      33   BIC:                             133.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
ER1            0.0037      0.176      0.021      0.9

In [9]:
reg2 = sm.OLS(endog=ticker_df['volume'], exog=ticker_df[['volume1', 'volume2', 'G', 'B', 'const']], missing='drop')
results2 = reg2.fit()

print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                 volume   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     1.819
Date:                Fri, 11 Dec 2020   Prob (F-statistic):              0.148
Time:                        00:04:35   Log-Likelihood:                -52.339
No. Observations:                  39   AIC:                             114.7
Df Residuals:                      34   BIC:                             123.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
volume1       -0.2524      0.168     -1.498      0.1

In [10]:
reg3 = sm.OLS(endog=ticker_df['volatility'], exog=ticker_df[['VOL1', 'VOL2', 'G', 'B', 'const']], missing='drop')
results3 = reg3.fit()

print(results3.summary())

                            OLS Regression Results                            
Dep. Variable:             volatility   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                 -0.086
Method:                 Least Squares   F-statistic:                    0.2514
Date:                Fri, 11 Dec 2020   Prob (F-statistic):              0.907
Time:                        00:04:36   Log-Likelihood:                 124.41
No. Observations:                  39   AIC:                            -238.8
Df Residuals:                      34   BIC:                            -230.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
VOL1           0.0120      0.172      0.070      0.9

In [11]:
with open('./regression results/'+'CSCO hourly'+' excess return.txt', 'w') as fh1:
    fh1.write(results.summary().as_text())
with open('./regression results/'+'CSCO hourly'+' volume.txt', 'w') as fh2:
    fh2.write(results2.summary().as_text())
with open('./regression results/'+'CSCO hourly'+' volatility.txt', 'w') as fh3:
    fh3.write(results3.summary().as_text())